# Variational Auto Encoder Notebook
Generative Algorithm that has been trained using FIRST Radio Sources. Makes use of the FRDEEP dataset of FIRST Radio Sources.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
from torchvision.utils import save_image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from torchsummary import summary
from MiraBest import MiraBest
from PIL import Image

In [2]:
import torch.nn as nn
import torch.nn.functional as F

In [3]:
import torch.optim as optim

Imports the images from the FRDEEP, we make use of the FIRST radio sources

In [4]:
from FRDEEP import FRDEEPF

In [38]:
def dataloader_first():
    transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize([0.5],[0.5])])
    
    trainset = FRDEEPF(root='./FIRST_data', train=True, download=True, transform=transform)  
    trainloader = torch.utils.data.DataLoader(trainset, shuffle=True, num_workers=2, batch_size=len(trainset))
    
    testset = FRDEEPF(root='./FIRST_data', train=False, download=True, transform=transform) 
    testloader = torch.utils.data.DataLoader(testset, shuffle=True, num_workers=2,batch_size = len(trainset))
    
    classes = ('FRI', 'FRII')
    
    # ---------------------------------------Train Data Array-------------------------------------------
    
    array_train= next(iter(trainloader))[0].numpy() # Training Datasets is loaded in numpy array
    
    array_label= next(iter(trainloader))[1].numpy() # Training Datasets labels is loaded in seperate numpy array
    
    augmented_data=np.zeros((19800,1,100,100))
    
    count=0
    
    for j in range(0,550):
        image_object=Image.fromarray(array_train[j,0,:,:])
        for i in range(0,36):
            rotated=image_object.rotate(i*10)
            imgarr = np.array(rotated)
            temp_img_array=imgarr[25:125,25:125]
            augmented_data[count,0,:,:]=temp_img_array
            count+=1
           
    augmented_data=(augmented_data-np.min(augmented_data))/(np.max(augmented_data)-np.min(augmented_data))
    
    X=augmented_data
    
    X_random_mix=np.take(X,np.random.permutation(X.shape[0]),axis=0,out=X);
    
    
    # ---------------------------------------Test Data Array-------------------------------------------
    array_test = next(iter(testloader))[0].numpy()
    
    augmented_data_test=np.zeros((50,1,100,100))
    
    count=0
 #   
    for j in range(0,50):
        image_object_test=Image.fromarray(array_test[j,0,:,:])
        for i in range(0,1):
            rotated_test=image_object_test.rotate(i*10)
            imgarr_test = np.array(rotated_test)
            temp_img_array_test=imgarr_test[25:125,25:125]
            augmented_data_test[count,0,:,:]=temp_img_array_test
            count+=1
           
    augmented_data_test=(augmented_data_test-np.min(augmented_data_test))/(np.max(augmented_data_test)-np.min(augmented_data_test))
    
    X_test=augmented_data_test
    
    X_random_mix_test=np.take(X_test,np.random.permutation(X_test.shape[0]),axis=0,out=X_test);
    
    
    
    
    return X_random_mix,X_random_mix_test

# Variational Auto Encoder Neural Network 
The Variational Auto Encoder consists of two neural network: i. An encoder and ii.a decoder.  
- The Encoder\
The enncoder consists of 5 layers and an input layer. The 100 by 100 pizels are reshaped to 10000 input features that are then reduced to 4096 in the first connected layer, then to 2048 features, then to 1024 features, to 512 features, 256 features and finally to the final 2 dimensional latent space z.

In [14]:
class VAE(nn.Module):
    def __init__(self, x_dim, h_dim1, h_dim2, h_dim3, h_dim4, h_dim5, z_dim):
        super(VAE, self).__init__()
        
        # encoder part
        self.fc1 = nn.Linear(x_dim, h_dim1) #x_dim=10000 to h_dim1=4096 
        self.fc2 = nn.Linear(h_dim1, h_dim2) #h_dim1=4096 to h_dim2=2048
        self.fc3 = nn.Linear(h_dim2, h_dim3) #h_dim2=2048 to h_dim3=1024
        self.fc4 = nn.Linear(h_dim3, h_dim4) #h_dim3=1024 to h_dim4=512
        self.fc5 = nn.Linear(h_dim4, h_dim5) #h_dim4=512 to h_dim5=256
        self.fc61 = nn.Linear(h_dim5, z_dim) #h_dim5=256 to z_dim=2
        self.fc62 = nn.Linear(h_dim5, z_dim) #h_dim5=256 to z_dim=2
        self.softplus = nn.Softplus()
        
        # decoder part
        self.fc7 = nn.Linear(z_dim, h_dim5) #z_dim=2 to h_dim5=256
        self.fc8 = nn.Linear(h_dim5, h_dim4) #h_dim5=256 to h_dim4=512
        self.fc9 = nn.Linear(h_dim4, h_dim3) #h_dim4=512 to h_dim3=1024
        self.fc10 = nn.Linear(h_dim3, h_dim2) #h_dim3=1024 to h_dim2=2048
        self.fc11 = nn.Linear(h_dim2, h_dim1) #h_dim2=2048 to h_dim1=4096
        self.fc12 = nn.Linear(h_dim1, x_dim)  #h_dim1=4096 to x_dim=10000
        self.softplus = nn.Softplus()
        self.sigmoid = nn.Sigmoid()
        
    def encoder(self, x):
        
        slope_param = 0.0001
        
        h = self.softplus(self.fc1(x))
        h = F.leaky_relu(self.fc2(h),slope_param)
        h = F.leaky_relu(self.fc3(h),slope_param)
        h = F.leaky_relu(self.fc4(h),slope_param)
        h = F.leaky_relu(self.fc5(h),slope_param)
        return self.fc61(h), self.fc62(h) # mu, log_var
    
    def sampling(self, mu, log_var):
        std = torch.exp(0.5*log_var)
        eps = torch.randn_like(std)
        return eps.mul(std).add_(mu) # return z sample
        
    def decoder(self, z):
        slope_param = 0.0001
        h = F.leaky_relu(self.fc7(z),slope_param)
        h = F.leaky_relu(self.fc8(h),slope_param)
        h = F.leaky_relu(self.fc9(h),slope_param)
        h = F.leaky_relu(self.fc10(h),slope_param)
        h = F.leaky_relu(self.fc11(h),slope_param)
        return F.sigmoid(self.fc12(h)) # Try to see what happens when we put a Softplus Here
    
    def forward(self, x):
        mu, log_var = self.encoder(x.view(-1, 10000))
        z = self.sampling(mu, log_var)
        return self.decoder(z), mu, log_var


In [15]:
# build model
vae = VAE(x_dim=10000, h_dim1= 4096, h_dim2=2048, h_dim3=1024, h_dim4=512, h_dim5=256, z_dim=2)

if torch.cuda.is_available():
    vae.cuda()

In [36]:
optimizer = optim.Adagrad(vae.parameters(), lr=0.3e-3)

# return reconstruction error + KL divergence losses
def loss_function(recon_x, x, mu, log_var):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 10000), reduction='sum')
    KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return BCE + KLD


In [39]:
X,X_test= dataloader_first()

Files already downloaded and verified
Files already downloaded and verified


In [46]:
def train(epoch):
    vae.train()
    train_loss = 0   
    data=torch.zeros(100, 1, 100, 100).cpu
    epoch_loss = 0
    for j in range(0,198):
        data = torch.from_numpy(X[j*100:(j+1)*100,:,:])
        data = data.cuda()
        optimizer.zero_grad()
        
        recon_batch, mu, log_var = vae(data.float())
        loss = loss_function(recon_batch.float(), data.float(), mu, log_var)
        
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
        #Testing Part:
    data_test = torch.from_numpy(X_test)
    data_test = data_test.cuda()
    recon_batch_test,mu_test,log_var_test = vae(data_test.float())
    test_loss = loss_function(recon_batch_test.float(), data_test.float(), mu_test, log_var_test).item()
        
        
    print("Training Epoch:"+str(epoch)+' Training Loss: '+str(train_loss/(198*100))+' Testing Loss: '+str(test_loss/50))

In [ ]:
for epoch in range(1, 100000):
    train(epoch)

Training Epoch:1 Training Loss: 135.77713946101642 Testing Loss: 128.789462890625
Training Epoch:2 Training Loss: 135.7536481021149 Testing Loss: 128.838857421875
Training Epoch:3 Training Loss: 135.74420005721277 Testing Loss: 128.117958984375
Training Epoch:4 Training Loss: 135.76207909169824 Testing Loss: 128.946357421875
Training Epoch:5 Training Loss: 135.75337846235794 Testing Loss: 129.151220703125
Training Epoch:6 Training Loss: 135.72132191051136 Testing Loss: 128.259091796875
Training Epoch:7 Training Loss: 135.73805989583335 Testing Loss: 128.4937890625
Training Epoch:8 Training Loss: 135.7175998263889 Testing Loss: 128.193994140625
Training Epoch:9 Training Loss: 135.70734153053976 Testing Loss: 129.2716796875
Training Epoch:10 Training Loss: 135.68647633562185 Testing Loss: 128.3391796875
Training Epoch:11 Training Loss: 135.68139747080176 Testing Loss: 128.8798046875
Training Epoch:12 Training Loss: 135.70294551964963 Testing Loss: 129.04337890625
Training Epoch:13 Traini

Training Epoch:102 Training Loss: 135.177102667298 Testing Loss: 128.8262109375
Training Epoch:103 Training Loss: 135.2064433889678 Testing Loss: 129.025458984375
Training Epoch:104 Training Loss: 135.17700012823548 Testing Loss: 128.804404296875
Training Epoch:105 Training Loss: 135.16783888691603 Testing Loss: 128.34677734375
Training Epoch:106 Training Loss: 135.18393327809343 Testing Loss: 128.96048828125
Training Epoch:107 Training Loss: 135.16604206123736 Testing Loss: 128.88298828125
Training Epoch:108 Training Loss: 135.13331917810922 Testing Loss: 128.839853515625
Training Epoch:109 Training Loss: 135.15892454821653 Testing Loss: 128.529443359375
Training Epoch:110 Training Loss: 135.15734325678662 Testing Loss: 129.147734375
Training Epoch:111 Training Loss: 135.12480843592172 Testing Loss: 129.298076171875
Training Epoch:112 Training Loss: 135.1424877683081 Testing Loss: 128.584111328125
Training Epoch:113 Training Loss: 135.1419163806029 Testing Loss: 128.75443359375
Traini

Training Epoch:201 Training Loss: 134.761865234375 Testing Loss: 128.42677734375
Training Epoch:202 Training Loss: 134.7711249704072 Testing Loss: 128.078525390625
Training Epoch:203 Training Loss: 134.73835010258838 Testing Loss: 128.784140625
Training Epoch:204 Training Loss: 134.72497795336173 Testing Loss: 128.774287109375
Training Epoch:205 Training Loss: 134.73868282433713 Testing Loss: 129.55431640625
Training Epoch:206 Training Loss: 134.73184885969064 Testing Loss: 128.9212109375
Training Epoch:207 Training Loss: 134.7166384548611 Testing Loss: 129.10228515625
Training Epoch:208 Training Loss: 134.7157875631313 Testing Loss: 128.774375
Training Epoch:209 Training Loss: 134.7487777679135 Testing Loss: 128.78359375
Training Epoch:210 Training Loss: 134.71532951586175 Testing Loss: 128.407900390625
Training Epoch:211 Training Loss: 134.69090810448233 Testing Loss: 129.475908203125
Training Epoch:212 Training Loss: 134.7170012133049 Testing Loss: 128.88462890625
Training Epoch:213

Training Epoch:301 Training Loss: 134.39103505760733 Testing Loss: 128.65181640625
Training Epoch:302 Training Loss: 134.39332090435607 Testing Loss: 128.74224609375
Training Epoch:303 Training Loss: 134.4016641512784 Testing Loss: 128.683955078125
Training Epoch:304 Training Loss: 134.36039659288195 Testing Loss: 129.471416015625
Training Epoch:305 Training Loss: 134.38780352351642 Testing Loss: 128.9965234375
Training Epoch:306 Training Loss: 134.3603604403409 Testing Loss: 129.278203125
Training Epoch:307 Training Loss: 134.36296189433398 Testing Loss: 128.89529296875
Training Epoch:308 Training Loss: 134.3582795730745 Testing Loss: 128.720986328125
Training Epoch:309 Training Loss: 134.36703129932133 Testing Loss: 129.264365234375
Training Epoch:310 Training Loss: 134.3721926294192 Testing Loss: 128.641943359375
Training Epoch:311 Training Loss: 134.3528447561553 Testing Loss: 128.71677734375
Training Epoch:312 Training Loss: 134.33276293205492 Testing Loss: 128.80681640625
Trainin

Training Epoch:400 Training Loss: 134.09384716303663 Testing Loss: 128.70439453125
Training Epoch:401 Training Loss: 134.06547210385102 Testing Loss: 128.6586328125
Training Epoch:402 Training Loss: 134.06930501302082 Testing Loss: 128.6662890625
Training Epoch:403 Training Loss: 134.08906471946023 Testing Loss: 129.3660546875
Training Epoch:404 Training Loss: 134.08195613360164 Testing Loss: 128.69806640625
Training Epoch:405 Training Loss: 134.07592304884784 Testing Loss: 128.692099609375
Training Epoch:406 Training Loss: 134.0736640329072 Testing Loss: 129.076103515625
Training Epoch:407 Training Loss: 134.0685368331755 Testing Loss: 128.738251953125
Training Epoch:408 Training Loss: 134.06558574021466 Testing Loss: 128.4219140625
Training Epoch:409 Training Loss: 134.0735054648043 Testing Loss: 128.91681640625
Training Epoch:410 Training Loss: 134.0625254498106 Testing Loss: 129.182021484375
Training Epoch:411 Training Loss: 134.04936760179925 Testing Loss: 128.859228515625
Trainin

Training Epoch:500 Training Loss: 133.81440538194445 Testing Loss: 129.73849609375
Training Epoch:501 Training Loss: 133.82542253590594 Testing Loss: 128.49939453125
Training Epoch:502 Training Loss: 133.81853081597222 Testing Loss: 129.187763671875
Training Epoch:503 Training Loss: 133.79161369554924 Testing Loss: 128.707861328125
Training Epoch:504 Training Loss: 133.79919936671402 Testing Loss: 128.7740234375
Training Epoch:505 Training Loss: 133.81683021622476 Testing Loss: 129.163388671875
Training Epoch:506 Training Loss: 133.79726725260417 Testing Loss: 129.0109765625
Training Epoch:507 Training Loss: 133.81953889480744 Testing Loss: 128.536962890625
Training Epoch:508 Training Loss: 133.81994249131944 Testing Loss: 129.088125
Training Epoch:509 Training Loss: 133.79369298453284 Testing Loss: 129.188076171875
Training Epoch:510 Training Loss: 133.7607953559028 Testing Loss: 128.837001953125
Training Epoch:511 Training Loss: 133.7908128649779 Testing Loss: 129.18478515625
Trainin

Training Epoch:600 Training Loss: 133.59389268663193 Testing Loss: 128.7295703125
Training Epoch:601 Training Loss: 133.5841981830019 Testing Loss: 128.664306640625
Training Epoch:602 Training Loss: 133.58100753630052 Testing Loss: 129.21396484375
Training Epoch:603 Training Loss: 133.5714000848327 Testing Loss: 129.317412109375
Training Epoch:604 Training Loss: 133.58670129024622 Testing Loss: 129.71873046875
Training Epoch:605 Training Loss: 133.58430762705177 Testing Loss: 128.317314453125
Training Epoch:606 Training Loss: 133.582594154435 Testing Loss: 128.1290625
Training Epoch:607 Training Loss: 133.58458249487057 Testing Loss: 128.399921875
Training Epoch:608 Training Loss: 133.5726817984533 Testing Loss: 129.095625
Training Epoch:609 Training Loss: 133.57664156210544 Testing Loss: 128.23494140625
Training Epoch:610 Training Loss: 133.5828082583649 Testing Loss: 129.06365234375
Training Epoch:611 Training Loss: 133.58869278724748 Testing Loss: 128.635498046875
Training Epoch:612

Sample the images from the latent space z, sample range between $-4.0 \leqslant z_1 \leqslant 4.0$ and $-4.0 \leqslant z_2 \leqslant 4.0$. Image are sampled at steps of 0.2 in both directions.

In [ ]:
with torch.no_grad():
    z = torch.randn(1600, 2).cuda()
    count_x=-4.0
    count_y=-4.0
    x=0
    y=0
    for i in range (0,40):
        for j in range (0,40):
            z[x,0]=count_x
            z[x,1]=count_y
            x=x+1
            count_y=count_y+0.2
        y=y+1
        count_x=count_x+0.2
        count_y=-4.0
        
    sample = vae.decoder(z).cuda()
    
    save_image(sample.view(1600, 1, 100, 100), 'sample_z_space_' +str(epoch)+'.png',nrow=40)

In [33]:
with torch.no_grad():
    z = torch.randn(100, 2).cuda()    
    sample = vae.decoder(z).cuda()
    save_image(sample.view(100, 1, 100, 100), 'sample_random_' +str(epoch)+'.png',nrow=10)